## Dash Canvas

dash-canvas - это модуль для аннотирования изображений и обработки изображений с помощью Dash. Он предоставляет как объект Dash Canvas для рисования и аннотаций на изображениях, так и набор служебных функций для обработки изображений с использованием аннотаций.

dash-canvas может использоваться в различных областях, где требуется взаимодействие пользователя с изображениями, таких как контроль качества в промышленности, идентификация и сегментация клеток или органов в естественных и медицинских науках, количественная оценка фаз в материалах и геонауках, построение обучающих наборов для машинного обучения и т.д.

In [14]:
from dash_canvas import DashCanvas
help(DashCanvas)

Help on class DashCanvas in module dash_canvas.DashCanvas:

class DashCanvas(dash.development.base_component.Component)
 |  DashCanvas(id=undefined, image_content=undefined, zoom=undefined, width=undefined, height=undefined, scale=undefined, tool=undefined, lineWidth=undefined, lineColor=undefined, goButtonTitle=undefined, filename=undefined, trigger=undefined, json_data=undefined, hide_buttons=undefined, **kwargs)
 |  
 |  A DashCanvas component.
 |  Canvas component for drawing on a background image and selecting
 |  regions.
 |  
 |  Keyword arguments:
 |  - id (string; optional): The ID used to identify this component in Dash callbacks
 |  - image_content (string; default ''): Image data string, formatted as png or jpg data string. Can be
 |  generated by utils.io_utils.array_to_data_string.
 |  - zoom (number; default 1): Zoom factor
 |  - width (number; default 500): Width of the canvas
 |  - height (number; default 500): Height of the canvas
 |  - scale (number; default 1): Scal

In [8]:
%%file 03_dash_opensource/canvas.py
from dash import Dash, html, dcc, Input, Output
from dash_canvas import DashCanvas
import networkx as nx
import matplotlib.pyplot as plt
from dash_canvas.utils import array_to_data_url

G = nx.karate_club_graph()
nx.draw(G, pos=nx.spring_layout(G, seed=42))
plt.savefig("03_dash_opensource/karate.jpg", dpi=300, bbox_inches='tight')
img = plt.imread("03_dash_opensource/karate.jpg")
image_content=array_to_data_url(img)

app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
app.layout = html.Div(
    [
        html.Div(
            children=[
                html.H3('На этом изображении можно рисовать!'),
                DashCanvas(
                    id="my-canvas",
                    tool='pencil', # line
                    width=1000,
                    lineColor="blue",
                    image_content=image_content
                ),
                dcc.Slider(
                    id='bg-width-slider',
                    min=2,
                    max=40,
                    step=1,
                    value=5
                ),
            ],
            className="six columns"
        ),
    ]
)

@app.callback(
    Output('my-canvas', 'lineWidth'),
    Input('bg-width-slider', 'value')
)
def _(value):
    return value

if __name__ == '__main__':
    app.run_server(debug=True)


Overwriting 03_dash_opensource/canvas.py


![Dash Canvas Karate](03_dash_opensource/canvas_karate.png)

## Dash Bootstrap

`dash-bootstrap-components` - это набор компонентов для plotly dash.

Чтобы использовать `dash-bootstrap-components`, нужно сделать две вещи:

1. Подключить таблицу стилей, совместимую с Bootstrap v5
2. Включите компоненты dash-bootstrap в макет приложения.

В компоненты `dbc` не входят стили или иконки, но `dbc` содержит ссылки на них в `dbc.themes` и `dbc.icons`. Для подключения только сетки можно использовать `dbc.themes.GRID`.

Bootstrap изначально оптимизирован для мобильных устройств. Чтобы обеспечить надлежащий рендеринг и сенсорное масштабирование для всех устройств, авторы требуют добавить тэг `meta`.
```html
<meta name="viewport" content="width=device-width, initial-scale=1">
```

Основные концепты bootstrap:
1. Контрольные точки (breakpoints) - определяют как адаптивный макет ведет себя в зависимости от размеров устройства
    - используют media queries для условной стилизации страниц
    - стратегия bootstrap: минимальный код для работы с маленькими устройствами, добавление стилей для бОльших размеров
    - доступные контрольные точки:
        - X-Small: < 576px;
        - small (sm): >= 576px;
        - medium (md): >= 768px;
        - large (lg): >= 992px;
        - extra large (xl): >= 1200px;
        - extra extra large (xxl): >= 1400px
    - контрольные точки выбирались так, чтобы удобно размещался контейнер с шириной, кратной 12;
    - контрольные точки влияют на все последующие точки (.col-sm-4 применяется к sm, md, lg, xl и xxl)
2. Контейнеры (containers) - фундаментальный строительный блок Bootstrap, который содержит, дополняет и выравнивает ваш контент в пределах данного устройства или области просмотра
    - необходимы для использования сетки bootstrap
    - имеются варианты:
        - .container: устанавливает max-width в пределах соответствующей контрольной точки
        - .container-fluid: устанавливает width: 100% для всех контрольных точек
        - .container-{breakpoint}: устанавливает width: 100% до указанной контрольной точки
    - выравнивают контент по центру (горизонтально и вертикально)
3. Сетка - механизм для построения любых макетов, использующий набор контейнеров, строк и столбцов для компоновки и выравнивания содержимого. Сетка построена с помощью flexbox и полностью адаптивна.
    - использует контрольные точки;
    - .row - это обертка для .col, благодаря которой контент столбцов визуально выравнивается по левому краю


В Dash у основых компонентов сетки есть следующие аргументы (перечислены не все):
- Container: 
    - id: str
    - fluid: bool
- Row:
    - id: str,
    - align: start, center, end, stretch, baseline - вертикальное выравнивание ячеек в строке
    - justify: горизонтальное выравнивание ячеек в строке
- Col:
    - id: str,
    - align: start, center, end, stretch, baseline - вертикальное выравнивание ячейки в родительской строке
    - lg, md, sm, xl, xs, xxl : int | dict : поведение ячейки на экране соответствующего размера 

In [19]:
%%file 03_dash_opensource/bootstrap_grid.py
import dash
# компоненты bootstrap
import dash_bootstrap_components as dbc

app = dash.Dash(
    # подключение стилей bootstrap
    external_stylesheets=[dbc.themes.BOOTSTRAP], # стили хранятся в CDN
    # альтернативно можно подключить по ссылке
    # external_stylesheets=["https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css"]
    # добавить тэг meta можно вот так:
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1"},
    ],
)

styles = {
    "Row": {
        "background-color": "olive",
        "border": "1px solid black",
        "height": "100px"
    },
    "Col": {
        "background-color": "pink",
        "border": "1px solid black"
    }
}

app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col('Col 1/3'), # .col
                dbc.Col('Col 2/3', width=6), # .col-6
                dbc.Col('Col 3/3'), # .col
            ],
            style=styles['Row']
        ),
        dbc.Row(
            [
                dbc.Col(
                    'Col 1/3', 
                    width={
                        "size": 3,
                        "offset": 1
                    }
                ), # .col-3 .offset-1
                dbc.Col('Col 2/3', width=6), # .col-6
                dbc.Col('Col 3/3'), # .col
            ],
            align="center",
            style=styles['Row']
        ),
        dbc.Row(
            [
                # если меньше md - каждая по 50%, если md+, то каждая по четверти экрана
                dbc.Col(
                    'Col 1/4', 
                    width=6, 
                    md=3, 
                    style=styles["Col"],
                    align="stretch"
                ), # .col-6 .col-md-3
                dbc.Col('Col 2/4', width=6, md=3, style=styles["Col"]), # .col-6 .col-md-3
                dbc.Col('Col 3/4', width=6, md=3, style=styles["Col"]), # .col-6 .col-md-3
                dbc.Col(
                    'Col 3/4', 
                    width=6, 
                    md=3, 
                    style=styles["Col"],
                    align="end"
                ), # .col-6 .col-md-3
            ],
            align="center",
            style=styles['Row']
        ),
        # fluid=True,
    ]
)

if __name__ == "__main__":
    app.run_server(debug=True)

Overwriting 03_dash_opensource/bootstrap.py


![Bootstrap Grid](03_dash_opensource/bs_grid.png)